<span style="font-size: 14px;"><b>Modeling Phases</b></span>

<span style="font-size: 14px;">Phase 0 - diagnostics</span>

<span style="font-size: 14px;">Phase 0a - FE in diagnostics&nbsp;</span> 

<span style="font-size: 14px;">Phase 1 - domain based models</span>

<span style="font-size: 14px;">Phase 1a - apply theories from SME/EDA</span>

<span style="font-size: 14px;">Phase 2 - bakeoff of algorithms</span>

<span style="font-size: 14px;">Phase 2a - permutations of features</span>

<span style="font-size: 14px;">Phase 3 - hyperparameter tuning</span>

<span style="font-size: 14px;">Phase 3a - cross validation/resampling</span>

In [1]:
use DW_202010WE_DS


: Msg 104455, Level 16, State 1, Line 1
USE statement is not supported to switch between databases. Use a new connection to connect to a different Database.

Total execution time: 00:00:00.380

In [8]:
--model fact table

ALTER view ADF.ModelfactTableFlightDelays
AS 
WITH
	 avg_d as (select AVG(DISTANCE)  avg_d  from ADF.FlightFact)
	 ,std_d as (select STDEV(DISTANCE) std_d from ADF.FlightFact)
SELECT
        ROUND(
            (
                (
                    (DISTANCE -31.0)/(4983.0-31.0))
                    * 
                    --num_bins+1
                    (100.0/20)
            )
                ,0
                --truncate
                ,1
                ) AS	DISTANCE_BIN_EQUAL_RANGE_06
    , CASE WHEN DISTANCE > 701 THEN 'LONG FLIGHT' ELSE 'SHORT FLIGHT' END DISTANCE_CUT_median
    , CASE 
        --top 70% NTILE<= 30 starts at 
        WHEN DISTANCE > 1020 THEN 'TOP FLIGHT DISTANCE'  
        --middle 40% starts at 
        WHEN DISTANCE > 447 THEN 'MEDIUM FLIGHT DISTANCE'  
        ELSE 'BOTTOM FLIGHT DISTANCE'
        END AS DISTANCE_BIN_APPROX_03
    ,NTILE(4) OVER(ORDER BY [DISTANCE]DESC) as DISTANCE_BIN_EQUAL_SIZE_04
    ,NTILE(100) OVER(ORDER BY [DISTANCE]DESC) as DISTANCE_BIN_EQUAL_SIZE_100
    ,(DISTANCE -31.0)/(4983.0-31.0) AS DISTANCE_SCALED_0_to_1
    ,100*((DISTANCE -31.0)/(4983.0-31.0))+1 AS DISTANCE_SCALED_1_to_100
    ,(DISTANCE - avg_d.avg_d)/std_d.std_d AS Z_SCORE
    ,D.*    
    ,F.*
FROM ADF.FlightFact F
INNER JOIN ADF.DimDate D
ON F.FlightDateKey = D.DateKey
	join std_d on 1 =1
	join avg_d on 1 =1
WHERE
    NAS_DELAY <= 150
    AND DEP_DELAY BETWEEN 15 AND 120

Commands completed successfully.

Total execution time: 00:00:00.150

In [2]:
drop view dbo.DistanceDeepDive

Commands completed successfully.

Total execution time: 00:00:00.192

In [3]:
SELECT AVG(DEP_DELAY) FROM ADF.ModelfactTableFlightDelays

: Msg 208, Level 16, State 1, Line 1
Invalid object name 'ADF.ModelfactTableFlightDelays'.

Total execution time: 00:00:00.054